# **Hello**

**This is a beginner friendly intermediate level notebook ;)**

**Please do comment and spread your knowledge. Enlighten me and others.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


# ***So here we start.***

In [2]:
#importing useful libraries
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [3]:
#Loading Datasets

X_full = pd.read_csv('../input/playground-series-s5e7/train.csv')
X_test = pd.read_csv('../input/playground-series-s5e7/test.csv')
X_test_og = X_test
"""X_full = X_full.dropna()"""

'X_full = X_full.dropna()'

In [4]:
X_test.isna().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

Some columns have categorical data and for now I don't have much expertize in dealing with categorical data, hence it is better to turn them into numerical ones.

In [5]:
#updating categorical data into binary terms

def turn_to_numeric(Dataframe, column, string):
    new = Dataframe[column] == string
    new += 0
    Dataframe[column] = new

# 1 means Yes, 0 implies No
"""turn_to_numeric(X_full, 'Stage_fear', 'No')
turn_to_numeric(X_full, 'Drained_after_socializing', "Yes")
turn_to_numeric(X_test, 'Stage_fear', 'No')
turn_to_numeric(X_test, 'Drained_after_socializing', 'Yes')
"""

X_full = X_full.sort_values(by = 'Stage_fear')
X_test = X_test.sort_values(by = 'Stage_fear')
X_full['Stage_fear'] = pd.Categorical(X_full['Stage_fear']).codes
X_test['Stage_fear'] = pd.Categorical(X_test['Stage_fear']).codes

X_full = X_full.sort_values(by = 'Drained_after_socializing')
X_test = X_test.sort_values(by = 'Drained_after_socializing')
X_full['Drained_after_socializing'] = pd.Categorical(X_full['Drained_after_socializing']).codes
X_test['Drained_after_socializing'] = pd.Categorical(X_test['Drained_after_socializing']).codes

X_full = X_full.sort_values(by = 'id')
X_test = X_test.sort_values(by = 'id')

# 1 means Exrovert and 0 refers to Introvert
turn_to_numeric(X_full, 'Personality', 'Extrovert')



Creating a new feature using drained after socializing and friends circle size

* An introvert can have a good friends circle size, but will get drained after socializing

There must be a relation between time spent alone and going out.
  

In [6]:
X_full['new1'] = X_full.Drained_after_socializing * X_full.Friends_circle_size
X_test['new1'] = X_test.Drained_after_socializing * X_test.Friends_circle_size

X_full['new2'] = X_full.Time_spent_Alone + X_full.Going_outside
X_test['new2'] = X_test.Time_spent_Alone + X_test.Going_outside

X_full['new3'] = X_full.Drained_after_socializing * X_full.Stage_fear
X_test['new3'] = X_test.Drained_after_socializing * X_test.Stage_fear



In [7]:
"""X_full = X_full.drop(['Post_frequency'], axis = 1)
X_test = X_test.drop(['Post_frequency'], axis = 1)"""

"X_full = X_full.drop(['Post_frequency'], axis = 1)\nX_test = X_test.drop(['Post_frequency'], axis = 1)"

In [8]:
X_full = X_full.fillna(500)
X_test = X_test.fillna(500)
X_full.sum()


id                           171560026.0
Time_spent_Alone                649390.0
Stage_fear                        2129.0
Social_event_attendance         681318.0
Going_outside                   801988.0
Drained_after_socializing         2913.0
Friends_circle_size             666703.0
Post_frequency                  717991.0
Personality                      13699.0
new1                            531121.0
new2                           1416049.0
new3                              1979.0
dtype: float64

Finally splitting the transformed data.

In [9]:
# Removing not needed columns before splitting the data

y = X_full["Personality"]
X = X_full.drop(["id", "Personality"], axis = 1)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=3)

# Imputation

Imputation for the NAN values.
Since the test data also contains NAN values, it is not a good idea to drop that data.

I will also check in some other version for putting 0 or something like that for NAN values. In that case, I will need to increase the weights of other values.

While there are ways by which we may create models without the need for imputation.
The link to the discussion where it is mentioned is : [How to go without imputing the values](https://www.kaggle.com/competitions/playground-series-s5e7/discussion/593875) suggestions provided by [Sylvester d'Almeida](https://www.kaggle.com/sylvesterdalmeida).



In [10]:
my_imputer = SimpleImputer()


cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

for col in cols_with_missing:
    X_train[col + '_was_missing'] = X_train[col].isnull()
    X_valid[col + '_was_missing'] = X_valid[col].isnull()

# Imputation

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns




# Creating the model

In [11]:
##### model creation
model = RandomForestClassifier(n_estimators = 350, random_state = 3)
model.fit(imputed_X_train, y_train)

RandomForestClassifier(n_estimators=350, random_state=3)

# Model Valiation

Model is created, let's now test it.

In [12]:
test_pred = model.predict(imputed_X_valid)
test_pred

array([1, 1, 1, ..., 0, 1, 0])

In [13]:
test_pred.mean()

0.7425101214574898

In [14]:
# In case of regression models such thing would have been needed.
# Since it is a classification problem.
for i in range(len(test_pred)):
    test_pred[i] = 1 if test_pred[i]>= 0.7 else 0

**Earlier when I used regression model**

(Our model made really nice predictions, but they were all values ranging from 0 to 1. While we just wanted only two values that are 0 and 1. So we converted those having value 0.5 and above to 1.)


Nice!! But we will require some thing to measure if our model really worked well

In [15]:
accuracy_score(y_valid, test_pred)

0.9665317139001349

The accuracy check can also be done manually as below

In [16]:
n = sum((y_valid - test_pred) !=0)
100 - (n / len(y_valid) * 100)

96.6531713900135

# Training on Whole Data

Now as we know how our model need to be, why not train it on whole train data

In [17]:
imputed_X_full = pd.DataFrame(my_imputer.fit_transform(X_full))
imputed_X_full.columns = X_full.columns
imputed_X_full = imputed_X_full.drop(['id', 'Personality'], axis = 1)

model.fit(imputed_X_full, y)

RandomForestClassifier(n_estimators=350, random_state=3)

Great!!! Now let's make our model work on real test data.

In [18]:
# taking a glance at the test data
X_test.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,new1,new2,new3
0,18524,3.0,0,7.0,4.0,0,6.0,500.0,0.0,7.0,0
1,18525,500.0,1,0.0,0.0,1,5.0,1.0,5.0,500.0,1
2,18526,3.0,0,5.0,6.0,0,15.0,9.0,0.0,9.0,0
3,18527,3.0,0,4.0,4.0,0,5.0,6.0,0.0,7.0,0
4,18528,9.0,1,1.0,2.0,1,1.0,1.0,1.0,11.0,1


The test data has some missing values, so let's impute them.

In [19]:
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))
imputed_X_test.columns = X_test.columns
imputed_X_test = imputed_X_test.drop(['id'], axis = 1)
final_prediction = model.predict(imputed_X_test)

final_prediction

array([1, 0, 1, ..., 1, 1, 0])

# Final Predictions

Final Predictions are made, now let's convert it into the asked format.

In [20]:
list = []
for i in range(len(final_prediction)):
    if final_prediction[i]>=0.7:
        list.append("Extrovert")
    else :
        list.append("Introvert")

output = pd.DataFrame({'id': X_test_og.id,
                       'Personality': list})
output.to_csv('submission.csv', index=False)

Awesome!! That's it, just one more line of code. (To see how our output looks like)

In [21]:
len(X_test)
output.groupby("Personality").count()

,id
Personality,
Extrovert,4615
Introvert,1560


Thank you for coming down till here.

If you reached here, do consider giving an upvote.